## DATA TEMİZLİĞİ VE HAZIRLIĞI 
Öncellikle işimize yarayacak olan kütüphaneleri import ettik.

In [1]:
import numpy as np
import requests
import json
import pandas as pd
from itertools import chain

[Hava Kalitesi İstasyon Ölçüm Sonuçları Web Servisini](https://data.ibb.gov.tr/dataset/hava-kalitesi-istasyon-olcum-sonuclari-web-servisi) kullanarak datayı çektik. json.dumps ifadesi Python objesini JSON stringine çevirdi ve ensure_ascii=False ifadesiyle Türkçe karakterleri kullanabilmemizi, sort_keys=True ile de sözlüğün keylerini sıralayabilmemizi sağladı. Daha sonra JSON stringe dönmüş ifademizi json.loads ile birlikte tekrar Python sözlüğüne çevirdik.

In [2]:
url = "https://api.ibb.gov.tr/havakalitesi/OpenDataPortalHandler/GetAQIStations"
JSONContent = requests.get(url).json()
content = json.dumps(JSONContent, indent = 4, sort_keys=True, ensure_ascii=False) 
resp = json.loads(content)

İlçelerin isimlerini ve IDlerini görebileceğimiz bir dictionary oluşturduk. 

In [3]:
ilce_dict = {}
for ilce in resp:
    ilce_dict.update({ilce["Name"]:ilce["Id"]})

İlçenin adresini, IDsini, point vektörlerini ve isimini görebileceğimiz bir data oluşturmak istedik. Bunun için başta boş bir data frame oluşturduk ve sütunları boş olarak atadık sonra bir for loopuyla bu sütunları valuelarıyla doldurduk. Böylece JSON datasından aldığımız bilgilerle bir data frame elde etmiş olduk.

In [4]:
ilce_info = pd.DataFrame()
ilce_info["Adress"] = None
ilce_info["Id"] = None
ilce_info["Location"] = None
ilce_info["Name"] = None
for i in range(len(resp)):
    ilce_info.at[i] = [resp[i]["Adress"], resp[i]["Id"], resp[i]["Location"], resp[i]["Name"]]

Adress sütununda her satırı belirli bir format haline getirmek için for döngüsü kurduk ve sütundaki ilçe isimleri aynı formda yazılmış oldu.

In [5]:
a = ilce_info["Adress"].apply(lambda row: row.split(" "))
i=0
for row in a:
    for eleman in row:
        if eleman != "İstanbul" and eleman != "Turkey" and len(eleman)>1:
            a = eleman + " Belediyesi"
            ilce_info.at[i, "Adress"] = a
    i = i+1
ilce_info.head()

,Adress,Id,Location,Name
0,Sarıyer Belediyesi,6b7a9840-1e13-4045-a79d-0f881c4852ad,POINT (29.024512004171349 41.100072371412381),Maslak
1,Esenler Belediyesi,914b525e-f275-41e1-b45b-118ada8f08c3,POINT (28.888071049660041 41.036871407562813),Esenler
2,Bahçelievler Belediyesi,6929a9ca-977d-4cb3-b1b5-26b95f4d4515,POINT (28.82703188207584 40.997387017164129),Yenibosna
3,Beylikdüzü Belediyesi,41f90c1f-570d-46cd-9beb-493d9162639a,POINT (28.612769803421966 40.971651116893213),Beylikdüzü
4,Ümraniye Belediyesi,377e1216-bcc7-42c0-aad8-4d5b3d602b78,POINT (29.161812932369855 41.0126908139832),Ümraniye 1


Point vektörlerin bulunduğu Location sütununu daha kullanışlı bir hale getirmek için apply methodunu kullandık. Daha öncesinde location sütununda point vektörler "POINT (29.024512004171349 41.100072371412381)" şeklinde görünüyordu. Biz apply methodu ile ilk 6 karakteri, "POINT "i, sildik. Daha sonra sayı ikililerinin arasına virgül ekledik. Böylece elde ettiğimiz yapı (29.024512004171349,41.100072371412381) şeklinde oldu. Daha sonrasında elde ettiğimiz bu veriyi csv dosyası halinde kaydettik.

In [6]:
ilce_info['Location'] = ilce_info['Location'].apply(lambda row: row[7:-1])
ilce_info['Location'] = ilce_info['Location'].apply(lambda row: row.split(" "))
ilce_info["Longitude"] = ilce_info['Location'].apply(lambda row: row[0])
ilce_info["Latitude"] = ilce_info['Location'].apply(lambda row: row[1])

In [7]:
ilce_info.to_csv("ilce_info1.csv")

Bir sonraki adımda, ilçelere göre ayrı ayrı verilmiş olan hava kalitesi sonuçlarını data olarak almamız gerekiyordu. Bunun için
her bir ilçe için, yani ilce_dict sözlüğünün her bir valuesu(IDsi) için, 2020ye dair hava kalitesi değerlerini aldık. Bu değerleri station_list adında bir listede topladık. 

In [8]:
station_list = []
for ilce in ilce_dict.values():   #ilce_dict'teki her bir id için
    JSONContent = requests.get("https://api.ibb.gov.tr/havakalitesi/OpenDataPortalHandler/GetAQIByStationId?StationId=" + ilce +
                               "&StartDate=01.01.2020%2000:00:00&EndDate=19.06.2020%2000:00:00").json() 
                                  #2020 hava kalitesi valuelarını aldık
    station_list.append([JSONContent]) 

Daha sonra bu listeyi data frame'e çevirdik. Ve ilce_info data frame'indeki Name ve Location valuelarını bu datasete ekledik. Locationdaki point vektörleri, daha sonra görselleştirme için kullanmak üzere, enlem ve boylam olarak ayırıp dataya ekledik.

In [9]:
dataset = pd.DataFrame(station_list)
dataset["Name"] = ilce_info["Name"].values
dataset["Location"] = ilce_info["Location"].values
dataset["Longitude"] = ilce_info["Longitude"].values
dataset["Latitude"] = ilce_info["Latitude"].values
dataset["Adress"] = ilce_info["Adress"].values
dataset.head() #datasetin ilk 5 satırı

,0,Name,Location,Longitude,Latitude,Adress
0,"[{'ReadTime': '2020-01-01T00:00:00', 'Concentr...",Maslak,"[29.024512004171349, 41.100072371412381]",29.024512004171349,41.100072371412381,Sarıyer Belediyesi
1,"[{'ReadTime': '2020-01-01T00:00:00', 'Concentr...",Esenler,"[28.888071049660041, 41.036871407562813]",28.888071049660041,41.036871407562813,Esenler Belediyesi
2,"[{'ReadTime': '2020-01-01T00:00:00', 'Concentr...",Yenibosna,"[28.82703188207584, 40.997387017164129]",28.82703188207584,40.997387017164129,Bahçelievler Belediyesi
3,"[{'ReadTime': '2020-01-01T00:00:00', 'Concentr...",Beylikdüzü,"[28.612769803421966, 40.971651116893213]",28.612769803421966,40.971651116893213,Beylikdüzü Belediyesi
4,"[{'ReadTime': '2020-01-01T00:00:00', 'Concentr...",Ümraniye 1,"[29.161812932369855, 41.0126908139832]",29.161812932369855,41.0126908139832,Ümraniye Belediyesi


Dataları her bir ilçe için aldıktan sonra bunu tek bir data içerisinde topladık.
Bunun için önce boş bir dictionary oluşturduk ve valueları ekledik. Daha sonrasında oluşan dictionarye diğer valueları eklemek için append kullandık. Aynı zamanda bazı ilçe verileri için hava kalitesi değeri(AQI) None'dı bu hataya sebep oluyordu, bu yüzden if yapılarına onu da dahil ettik. Bu datayı csv dosyasına kaydettik.

In [10]:
dataframe = pd.DataFrame()
for i in range(len(dataset)):            #28 tane ilçe için 
    a_dict = {}
    for a_ilce in dataset.iloc[i][0]:    #her bir satıra bakıyor.
        if a_dict=={}:                   #ilk satırı oluştururken dictionary boş olacak.  
            if a_ilce["AQI"] != None: 
                a_dict["ReadTime"] = [a_ilce["ReadTime"]]
                a_dict["AQIIndex"] = [a_ilce["AQI"]["AQIIndex"]]
                a_dict["Name"] = [dataset.at[i,'Name']]
                a_dict["Location"] = [dataset.at[i,'Location']]
                a_dict["Longitude"] = [dataset.at[i,'Longitude']]
                a_dict["Latitude"] = [dataset.at[i,'Latitude']]
                a_dict["Adress"] = [dataset.at[i,'Adress']]
                
            else:
                a_dict["ReadTime"] = [a_ilce["ReadTime"]]
                a_dict["AQIIndex"] = ["None"]
                a_dict["Name"] = [dataset.at[i,'Name']]
                a_dict["Location"] = [dataset.at[i,'Location']]
                a_dict["Longitude"] = [dataset.at[i,'Longitude']]
                a_dict["Latitude"] = [dataset.at[i,'Latitude']]
                a_dict["Adress"] = [dataset.at[i,'Adress']]
        elif a_ilce["AQI"] != None:
            a_dict["ReadTime"].append(a_ilce["ReadTime"])
            a_dict["AQIIndex"].append(a_ilce["AQI"]["AQIIndex"])
            a_dict["Name"].append(dataset.at[i,'Name'])
            a_dict["Location"].append(dataset.at[i,'Location'])
            a_dict["Longitude"].append(dataset.at[i,'Longitude'])
            a_dict["Latitude"].append(dataset.at[i,'Latitude'])
            a_dict["Adress"].append(dataset.at[i,'Adress'])
        else:
            a_dict["ReadTime"].append(a_ilce["ReadTime"])
            a_dict["AQIIndex"].append("None")
            a_dict["Name"].append(dataset.at[i,'Name'])
            a_dict["Location"].append(dataset.at[i,'Location'])
            a_dict["Longitude"].append(dataset.at[i,'Longitude'])
            a_dict["Latitude"].append(dataset.at[i,'Latitude'])
            a_dict["Adress"].append(dataset.at[i,'Adress'])
    if dataframe.empty:                   #ilk data frame'i oluşturduk
        dataframe=pd.DataFrame(a_dict)
    else:                                 #oluşan data frame'e ekleme yaptık.
        dataframe2 = pd.DataFrame(a_dict)  
        dataframe = dataframe.append(dataframe2, ignore_index = True)
dataframe.to_csv('dataframe1.csv')